In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Difference"
output_dir = r"C:\Users\llccl\Downloads\1.Difference"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%|   | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%| | 1/484 [00:44<6:02:00, 44
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   0%| | 2/484 [01:45<7:12:48, 53
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 3/484 [02:50<7:53:52, 59
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 4/484 [04:18<9:23:34, 70
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 5/484 [05:20<8:58:52, 67
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif:   1%| | 6/484 [06:08<8:05:56, 61
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 0/484 [00:00<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 1/484 [00:43<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   0%| | 2/484 [01:30<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 3/484 [02:16<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 4/484 [03:20<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 5/484 [04:11<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 6/484 [04:49<
Processing Masks for reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif:   1%| | 7/484 [05:32<
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1DN_2016-2020_2000-2005_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 0/484 [00:00<?, ?it/
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 1/484 [00:44<5:59:16
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   0%| | 2/484 [01:32<6:12:17
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 3/484 [02:20<6:18:34
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 4/484 [03:26<7:16:14
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 5/484 [04:18<7:09:49
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 6/484 [04:56<6:24:38
Processing Masks for reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif:   1%| | 7/484 [05:38<6:08:52
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1D_2016-2020_2000-2005_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 0/484 [00:00<?, ?it/
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 1/484 [00:45<6:03:29
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   0%| | 2/484 [01:32<6:13:58
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 3/484 [02:20<6:16:24
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 4/484 [03:29<7:27:56
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 5/484 [04:29<7:37:47
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 6/484 [05:08<6:46:09
Processing Masks for reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif:   1%| | 7/484 [05:52<6:28:35
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\reprojected_corrected_MOYD21A1N_2016-2020_2000-2005_mean_Average_transition_matrix.csv


In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Linear Trend"
output_dir = r"C:\Users\llccl\Downloads\2.LinearTrend"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [00:49<6:40:52, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [01:49<7:26:22, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [02:43<7:19:03, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [03:58<8:22:59, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [04:57<8:11:33, 
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [05:43<7:26:25, 
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 0/484 [00:0
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 1/484 [00:4
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   0%| | 2/484 [01:3
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 3/484 [02:2
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 4/484 [03:3
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 5/484 [04:2
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 6/484 [05:0
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif:   1%| | 7/484 [05:4
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1DN_2000-2020_LinearTrend_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%|  | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [00:43<5:47:32, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [01:30<6:04:23, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [02:17<6:09:47, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [03:25<7:18:51, 5
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [04:20<7:19:44, 5
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [05:00<6:37:10, 4
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif:   1%| | 7/484 [05:44<6:21:05, 4
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1D_2000-2020_LinearTrend_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%|  | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%| | 1/484 [00:44<5:57:43, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   0%| | 2/484 [01:32<6:12:56, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 3/484 [02:20<6:17:41, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 4/484 [03:28<7:23:08, 5
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 5/484 [04:22<7:17:23, 5
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 6/484 [05:00<6:32:33, 4
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif:   1%| | 7/484 [05:43<6:15:18, 4
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\reprojected_corrected_MOYD21A1N_2000-2020_LinearTrend_Average_transition_matrix.csv


In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bars
from rasterio.transform import rowcol, xy

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2000 to 2020 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Corrected\Mean"
output_dir = r"C:\Users\llccl\Downloads\3.Mean"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get the list of LST files for progress tracking
lst_files = [f for f in os.listdir(lst_dir) if f.endswith(".tif")]

# Process all LST files with tqdm progress bar
for lst_filename in tqdm(lst_files, desc="Processing LST Files"):
    lst_path = os.path.join(lst_dir, lst_filename)
    output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
    
    # Initialize a 22x22 matrix to store mean values
    transition_matrix = np.full((22, 22), np.nan)
    
    # Open the LST image
    with rasterio.open(lst_path) as lst_ds:
        lst_data = lst_ds.read(1)
        lst_transform = lst_ds.transform
        lst_crs = lst_ds.crs

        # Iterate over each mask file with a progress bar
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_Mask.tif")]
        for filename in tqdm(mask_files, desc=f"Processing Masks for {lst_filename}", leave=False):
            # Parse the class transition from the filename
            parts = filename.split('_')
            class_from = int(parts[0].replace("Class", ""))
            class_to = int(parts[2].replace("Class", ""))
            
            # Check if class_from and class_to are in the specified classes
            if class_from in class_to_index and class_to in class_to_index:
                # Get corresponding indices for matrix placement
                from_idx = class_to_index[class_from]
                to_idx = class_to_index[class_to]
                
                # Open the mask image
                mask_path = os.path.join(mask_dir, filename)
                with rasterio.open(mask_path) as mask_ds:
                    mask_data = mask_ds.read(1)
                    mask_transform = mask_ds.transform
                    mask_crs = mask_ds.crs
                    
                    # Check if CRS matches
                    if mask_crs != lst_crs:
                        print(f"CRS mismatch: {mask_crs} (mask) vs {lst_crs} (LST). Skipping {filename}.")
                        continue
                    
                    # Find coordinates of mask pixels with value 1
                    mask_indices = np.argwhere(mask_data == 1)
                    mask_coords = [xy(mask_transform, row, col) for row, col in mask_indices]
                    
                    # Map mask coordinates to LST pixel indices
                    lst_indices = [rowcol(lst_transform, x, y) for x, y in mask_coords]
                    lst_values = []
                    
                    for row, col in lst_indices:
                        if 0 <= row < lst_data.shape[0] and 0 <= col < lst_data.shape[1]:
                            value = lst_data[row, col]
                            if not np.isnan(value):  # Exclude NaN values
                                lst_values.append(value)
                    
                    # Remove NaN and calculate statistics
                    lst_values = np.array(lst_values)
                    if lst_values.size > 0:
                        q1 = np.percentile(lst_values, 25)
                        q3 = np.percentile(lst_values, 75)
                        filtered_values = lst_values[(lst_values >= q1) & (lst_values <= q3)]
                        
                        if filtered_values.size > 0:
                            mean_value = np.mean(filtered_values)
                            transition_matrix[from_idx, to_idx] = mean_value
    
    # Convert matrix to DataFrame and save to CSV
    transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
    transition_df.to_csv(output_csv, index=True)
    print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processing LST Files:   0%|                                                                      | 0/4 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%|        | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%| | 1/484 [00:54<7:16:23, 54.21s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   0%| | 2/484 [01:51<7:28:41, 55.85s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 3/484 [02:45<7:21:19, 55.05s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 4/484 [03:58<8:17:22, 62.17s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 5/484 [04:51<7:50:02, 58.88s/
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif:   1%| | 6/484 [05:33<7:04:15, 53.25s/
Processing Masks for reprojected_correct

Processed reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1DN_2000-2020_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 0/484 [00:00<?, ?i
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 1/484 [00:42<5:42:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   0%| | 2/484 [01:28<5:58:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 3/484 [02:14<6:01:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 4/484 [03:22<7:15:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 5/484 [04:17<7:15:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 6/484 [04:53<6:23:
Processing Masks for reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif:   1%| | 7/484 [05:34<6:04:
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1DN_2000-2020_mean_RegressCombination_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%|         | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%| | 1/484 [00:42<5:43:26, 42.66s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   0%| | 2/484 [01:32<6:16:15, 46.84s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 3/484 [02:21<6:23:14, 47.81s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 4/484 [03:35<7:44:09, 58.02s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 5/484 [04:29<7:33:10, 56.76s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 6/484 [05:07<6:41:37, 50.41s/i
Processing Masks for reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif:   1%| | 7/484 [05:52<6:26:46, 48.65s/i
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1D_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1D_2000-2020_mean_Average_transition_matrix.csv



Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%|         | 0/484 [00:00<?, ?it/s]
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%| | 1/484 [00:44<5:54:48, 44.08s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   0%| | 2/484 [01:36<6:31:09, 48.69s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 3/484 [02:28<6:43:57, 50.39s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 4/484 [03:41<7:54:55, 59.37s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 5/484 [04:39<7:48:43, 58.71s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 6/484 [05:18<6:54:23, 52.02s/i
Processing Masks for reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif:   1%| | 7/484 [06:06<6:44:39, 50.90s/i
Processing Masks for reprojected_correc

Processed reprojected_corrected_MOYD21A1N_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\reprojected_corrected_MOYD21A1N_2000-2020_mean_Average_transition_matrix.csv
